In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mmcls

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.8/648.8 kB 10.9 MB/s eta 0:00:00


In [3]:
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())


2.0.0+cu118
True


In [4]:
# Clone mmcls repository and checkout to the 1.x branch
!git clone -b 1.x https://github.com/open-mmlab/mmclassification.git
%cd mmclassification/
# Install MMClassification from source by mim
!pip install openmim
!mim install -e . 

Cloning into 'mmclassification'...
remote: Enumerating objects: 16220, done.
remote: Counting objects: 100% (1051/1051), done.
remote: Compressing objects: 100% (539/539), done.
remote: Total 16220 (delta 620), reused 814 (delta 500), pack-reused 15169
Receiving objects: 100% (16220/16220), 13.07 MiB | 19.39 MiB/s, done.
Resolving deltas: 100% (11251/11251), done.
/content/mmclassification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Obtaining file:///content/mmclassification
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.1/372.1 kB 7.5 MB/s eta 0:00:

In [5]:
import mmcls
from PIL import Image
import os
import shutil
import pandas as pd
import torch.optim as optim
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler


In [ ]:
# đường dẫn đến thư mục chứa ảnh
train_dir = '/content/drive/MyDrive/data/train'

# đường dẫn đến file csv chứa nhãn
label_file = '/content/drive/MyDrive/data/train.csv'

# đường dẫn đến thư mục train sẽ được tạo ra
trains_dir = '/content/drive/MyDrive/data/sapxeptrain'

# tạo thư mục train nếu chưa tồn tại
if not os.path.exists(trains_dir):
    os.makedirs(trains_dir)

# đọc file csv chứa nhãn
labels_df = pd.read_csv(label_file)

# lặp qua các dòng trong file csv và sao chép ảnh vào thư mục train tương ứng
for index, row in labels_df.iterrows():
    img_filename = row['image'] + '.jpg'
    img_label = row.drop('image').astype(int).idxmax()

    label_dir = os.path.join(trains_dir, img_label)
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
    src_path = os.path.join(train_dir, img_filename)
    dst_path = os.path.join(label_dir, img_filename)
    shutil.copyfile(src_path, dst_path)

In [ ]:
# đường dẫn đến thư mục chứa ảnh
valid_dir = '/content/drive/MyDrive/data/valid'

# đường dẫn đến file csv chứa nhãn
label_file = '/content/drive/MyDrive/data/valid.csv'

# đường dẫn đến thư mục sẽ được tạo ra
valids_dir = '/content/drive/MyDrive/data/sapxepvalid'

# tạo thư mục nếu chưa tồn tại
if not os.path.exists(valids_dir):
    os.makedirs(valids_dir)

# đọc file csv chứa nhãn
labels_df = pd.read_csv(label_file)

# lặp qua các dòng trong file csv và sao chép ảnh vào thư mục train tương ứng
for index, row in labels_df.iterrows():
    img_filename = row['image'] + '.jpg'
    img_label = row.drop('image').astype(int).idxmax()

    label_dir = os.path.join(valids_dir, img_label)
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
    src_path = os.path.join(valid_dir, img_filename)
    dst_path = os.path.join(label_dir, img_filename)
    shutil.copyfile(src_path, dst_path)

In [ ]:
# đường dẫn đến thư mục chứa ảnh
test_dir = '/content/drive/MyDrive/data/test'

# đường dẫn đến file csv chứa nhãn
label_file = '/content/drive/MyDrive/data/test.csv'

# đường dẫn đến thư mục sẽ được tạo ra
tests_dir = '/content/drive/MyDrive/data/sapxeptest'

# tạo thư mục nếu chưa tồn tại
if not os.path.exists(tests_dir):
    os.makedirs(tests_dir)

# đọc file csv chứa nhãn
labels_df = pd.read_csv(label_file)

# lặp qua các dòng trong file csv và sao chép ảnh vào thư mục train tương ứng
for index, row in labels_df.iterrows():
    img_filename = row['image'] + '.jpg'
    img_label = row.drop('image').astype(int).idxmax()

    label_dir = os.path.join(tests_dir, img_label)
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
    src_path = os.path.join(test_dir, img_filename)
    dst_path = os.path.join(label_dir, img_filename)
    shutil.copyfile(src_path, dst_path)

In [ ]:
%%writefile /content/drive/MyDrive/AI1/config.py
_base_ = [
    '/content/mmclassification/configs/_base_/models/efficientnet_v2/efficientnetv2_l.py',
    '/content/mmclassification/configs/_base_/schedules/imagenet_bs256_coslr.py',
    '/content/mmclassification/configs/_base_/default_runtime.py'
]

# ---- model settings ----
# Here we use init_cfg to load pre-trained model.
# In this way, only the weights of backbone will be loaded.
# And modify the num_classes to match our dataset.

model = dict(
    type='ImageClassifier',
    backbone=dict(type='EfficientNetV2', arch='l'),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=7,
        in_channels=1280,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, 5),
    ))

# ---- data settings ----
# We re-organized the dataset as `CustomDataset` format.
dataset_type = 'CustomDataset'
data_preprocessor = dict(
    mean=[127.5, 127.5, 127.5],
    std=[127.5, 127.5, 127.5],
    # convert image from BGR to RGB
    to_rgb=True,
)

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', scale=224),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    dict(type='PackClsInputs'),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='ResizeEdge', scale=256, edge='short'),
    dict(type='CenterCrop', crop_size=224),
    dict(type='PackClsInputs'),
]

train_dataloader = dict(
    batch_size=16,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_prefix='/content/drive/MyDrive/data/sapxeptrain',
        classes=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'],
        pipeline=train_pipeline,
    ),
    sampler=dict(type='DefaultSampler', shuffle=True),
)

val_dataloader = dict(
    batch_size=16,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_prefix='/content/drive/MyDrive/data/sapxepvalid',
        classes=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'],
        pipeline=test_pipeline,
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
)

test_dataloader = dict(
    batch_size=16,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_prefix='/content/drive/MyDrive/data/sapxeptest',
        classes=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'],
        pipeline=test_pipeline,
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
)

# Specify the evaluation metric for validation and testing.
val_evaluator = dict(type='Accuracy', topk=1)
test_evaluator = val_evaluator

# ---- schedule settings ----
# Usually in fine-tuning, we need a smaller learning rate and less training epochs.
# Specify the learning rate
optim_wrapper = dict(
    optimizer=dict(type='SGD', lr=0.1, momentum=0.9, weight_decay=0.0001))
# Set the learning rate scheduler
param_scheduler = dict(
    type='MultiStepLR', by_epoch=True, milestones=[30, 60, 90], gamma=0.1)

train_cfg = dict(by_epoch=True, max_epochs=4, val_interval=1)
val_cfg = dict()
test_cfg = dict()

default_hooks = dict(logger=dict(interval=50))

randomness = dict(seed=42, deterministic=False)

In [9]:
#!python /content/mmclassification/tools/train.py /content/drive/MyDrive/AI1/config.py
!python tools/train.py \
   /content/drive/MyDrive/AI1/config.py \
  --work-dir /content/drive/MyDrive/data/work_dir

05/12 14:31:50 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 42
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

In [10]:
!python tools/test.py /content/drive/MyDrive/AI1/config.py /content/drive/MyDrive/data/work_dir/epoch_4.pth --out result.pkl

05/12 15:05:22 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 42
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

In [11]:
import mmengine

results = mmengine.load("result.pkl")
# Output the first samples' ground truth and prediction.
print('Ground truth:', results[0]['gt_label'])
print('Prediction:', results[0]['pred_label'])

Ground truth: {'label': tensor([0])}
Prediction: {'score': tensor([0.1006, 0.1064, 0.1009, 0.0372, 0.0654, 0.5750, 0.0143]), 'label': tensor([5])}


In [12]:
!python demo/image_demo.py /content/drive/MyDrive/data/sapxeptest/DF/ISIC_0035012.jpg /content/drive/MyDrive/AI1/config.py --checkpoint /content/drive/MyDrive/data/work_dir/epoch_4.pth

Loads checkpoint by local backend from path: /content/drive/MyDrive/data/work_dir/epoch_4.pth
Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   
{
  "pred_label": 5,
  "pred_score": 0.5903218984603882,
  "pred_class": "DF"
}
